# Extracción de texto con base en el contexto

## Importar librerías

In [ ]:
import pathlib
import re
from functools import partial
from typing import Generator

from bs4 import BeautifulSoup, Doctype, NavigableString, SoupStrainer, Tag
from dotenv import load_dotenv
from html2text import HTML2Text
from IPython.core.display import Markdown
from langchain.document_loaders import DocugamiLoader, RecursiveUrlLoader

load_dotenv()

## Web

### Dataset y función de utilidad

In [ ]:
doc_url = "https://python.langchain.com/docs/get_started/quickstart"

load_documents = partial(
    RecursiveUrlLoader,
    url=doc_url,
    max_depth=3,
    prevent_outside=True,
    check_response_status=True,
)

### Extracción de texto sin tener en cuenta el contexto

La primera aproximación para extraer texto de una página web es simplemente obtener el texto de todos los elementos de la página.

### Extracción de texto teniendo un poco de contexto

El texto de la documentación de `Langchain` está escrito en `Markdown` y por lo tanto, tiene una estructura que puede ser aprovechada para extraer el texto de manera más precisa.

Utilicemos una librería que nos permita convertir el texto de `HTML` a `Markdown` y así poder extraer el texto de manera más precisa.

### Extracción de texto teniendo en cuenta el contexto

Si bien, cuando utilizamos una libreraía para convertir el texto de `HTML` a `Markdown` pudimos extraer el texto de manera más precisa, aún hay algunos casos en los que no se logra extraer el texto de manera correcta.

Es aquí donde entra en juego el dominio del problema. Con base en el conocimiento que tenemos del problema, podemos crear una función que nos permita extraer el texto de manera más precisa.

Imagina que `langchain_docs_extractor` es como un obrero especializado en una fábrica cuyo trabajo es transformar materias primas (documentos HTML) en un producto terminado (un string limpio y formateado). Este obrero usa una herramienta especial, `get_text`, como una máquina para procesar las materias primas en piezas utilizables, examinando cada componente de la materia prima **pieza por pieza**, y usa el mismo proceso repetidamente (**recursividad**) para descomponer los componentes en su forma más simple. Al final, ensambla todas las piezas procesadas en un producto completo y hace algunos refinamientos finales antes de que el producto salga de la fábrica.

In [ ]:
def langchain_docs_extractor(
    html: str,
    include_output_cells: bool,
    path_url: str | None = None,
) -> str:
    soup = BeautifulSoup(
        html,
        "lxml",
        parse_only=SoupStrainer(name="article"),
    )

    # Remove all the tags that are not meaningful for the extraction.
    SCAPE_TAGS = ["nav", "footer", "aside", "script", "style"]
    [tag.decompose() for tag in soup.find_all(SCAPE_TAGS)]

    # get_text() method returns the text of the tag and all its children.
    def get_text(tag: Tag) -> Generator[str, None, None]:
        for child in tag.children:
            if isinstance(child, Doctype):
                continue

            if isinstance(child, NavigableString):
                yield child.get_text()
            elif isinstance(child, Tag):
                if child.name in ["h1", "h2", "h3", "h4", "h5", "h6"]:
                    text = child.get_text(strip=False)

                    if text == "API Reference:":
                        yield f"> **{text}**\n"
                        ul = child.find_next_sibling("ul")
                        if ul is not None and isinstance(ul, Tag):
                            ul.attrs["api_reference"] = "true"
                    else:
                        yield f"{'#' * int(child.name[1:])} "
                        yield from child.get_text(strip=False)

                        if path_url is not None:
                            link = child.find("a")
                            if link is not None:
                                yield f" [](/{path_url}/{link.get('href')})"
                        yield "\n\n"
                elif child.name == "a":
                    yield f"[{child.get_text(strip=False)}]({child.get('href')})"
                elif child.name == "img":
                    yield f"![{child.get('alt', '')}]({child.get('src')})"
                elif child.name in ["strong", "b"]:
                    yield f"**{child.get_text(strip=False)}**"
                elif child.name in ["em", "i"]:
                    yield f"_{child.get_text(strip=False)}_"
                elif child.name == "br":
                    yield "\n"
                elif child.name == "code":
                    parent = child.find_parent()
                    if parent is not None and parent.name == "pre":
                        classes = parent.attrs.get("class", "")

                        language = next(
                            filter(lambda x: re.match(r"language-\w+", x), classes),
                            None,
                        )
                        if language is None:
                            language = ""
                        else:
                            language = language.split("-")[1]

                        if language in ["pycon", "text"] and not include_output_cells:
                            continue

                        lines: list[str] = []
                        for span in child.find_all("span", class_="token-line"):
                            line_content = "".join(
                                token.get_text() for token in span.find_all("span")
                            )
                            lines.append(line_content)

                        code_content = "\n".join(lines)
                        yield f"```{language}\n{code_content}\n```\n\n"
                    else:
                        yield f"`{child.get_text(strip=False)}`"

                elif child.name == "p":
                    yield from get_text(child)
                    yield "\n\n"
                elif child.name == "ul":
                    if "api_reference" in child.attrs:
                        for li in child.find_all("li", recursive=False):
                            yield "> - "
                            yield from get_text(li)
                            yield "\n"
                    else:
                        for li in child.find_all("li", recursive=False):
                            yield "- "
                            yield from get_text(li)
                            yield "\n"
                    yield "\n\n"
                elif child.name == "ol":
                    for i, li in enumerate(child.find_all("li", recursive=False)):
                        yield f"{i + 1}. "
                        yield from get_text(li)
                        yield "\n\n"
                elif child.name == "div" and "tabs-container" in child.attrs.get(
                    "class", [""]
                ):
                    tabs = child.find_all("li", {"role": "tab"})
                    tab_panels = child.find_all("div", {"role": "tabpanel"})
                    for tab, tab_panel in zip(tabs, tab_panels):
                        tab_name = tab.get_text(strip=True)
                        yield f"{tab_name}\n"
                        yield from get_text(tab_panel)
                elif child.name == "table":
                    thead = child.find("thead")
                    header_exists = isinstance(thead, Tag)
                    if header_exists:
                        headers = thead.find_all("th")
                        if headers:
                            yield "| "
                            yield " | ".join(header.get_text() for header in headers)
                            yield " |\n"
                            yield "| "
                            yield " | ".join("----" for _ in headers)
                            yield " |\n"

                    tbody = child.find("tbody")
                    tbody_exists = isinstance(tbody, Tag)
                    if tbody_exists:
                        for row in tbody.find_all("tr"):
                            yield "| "
                            yield " | ".join(
                                cell.get_text(strip=True) for cell in row.find_all("td")
                            )
                            yield " |\n"

                    yield "\n\n"
                elif child.name in ["button"]:
                    continue
                else:
                    yield from get_text(child)

    joined = "".join(get_text(soup))
    return re.sub(r"\n\n+", "\n\n", joined).strip()

El archivo de salida es ahora en formato Markdown, lo que permite visualizarlo en cualquier editor de texto o en GitHub, ofreciendo una estructura de la información más clara y accesible. Esta organización permite realizar cortes de texto con mayor precisión, facilitando así la obtención de información más pertinente y relevante.

In [ ]:
Markdown(docs_with_data_context[0].page_content)

## PDF / DOCX / DOC

### Dataset de prueba

En este ejemplo, vamos a emplear algunos archivos de muestra proporcionados por [Docugami](https://www.docugami.com/). Dichos archivos representan el producto de la extracción de texto de documentos auténticos, en particular, de archivos PDF relativos a contratos de arrendamiento comercial.

In [ ]:
lease_data_dir = pathlib.Path("../data/docugami/commercial_lease")
lease_files = list(lease_data_dir.glob("*.xml"))
lease_files

Ahora, carguemos los documentos de muestra y veamos qué propiedades tienen.

La metadata obtenida del documento incluye los siguientes elementos:

- `id`, `source_id` y `name`: Estos campos identifican de manera unívoca al documento y al fragmento de texto que se ha extraído de él.
- `xpath`: Es el `XPath` correspondiente dentro de la representación XML del documento. Se refiere específicamente al fragmento extraído. Este campo es útil para referenciar directamente las citas del fragmento real dentro del documento XML.
- `structure`: Incluye los atributos estructurales del fragmento, tales como `p`, `h1`, `div`, `table`, `td`, entre otros. Es útil para filtrar ciertos tipos de fragmentos, en caso de que el usuario los requiera.
- `tag`: Representa la etiqueta semántica para el fragmento. Se genera utilizando diversas técnicas, tanto generativas como extractivas, para determinar el significado del fragmento en cuestión.

`Docugami` también posee la capacidad de asistir en la extracción de metadatos específicos para cada `chunk` o fragmento de nuestros documentos. A continuación, se presenta un ejemplo de cómo se extraen y representan estos metadatos:

```json
{
    'xpath': '/docset:OFFICELEASEAGREEMENT-section/docset:OFFICELEASEAGREEMENT/docset:LeaseParties',
    'id': 'v1bvgaozfkak',
    'source': 'TruTone Lane 2.docx',
    'structure': 'p',
    'tag': 'LeaseParties',
    'Lease Date': 'April 24 \n\n ,',
    'Landlord': 'BUBBA CENTER PARTNERSHIP',
    'Tenant': 'Truetone Lane LLC',
    'Lease Parties': 'Este ACUERDO DE ARRENDAMIENTO DE OFICINA (el "Contrato") es celebrado por y entre BUBBA CENTER PARTNERSHIP ("Arrendador"), y Truetone Lane LLC, una compañía de responsabilidad limitada de Delaware ("Arrendatario").'
}
```

Los metadatos adicionales, como los mostrados arriba, pueden ser extremadamente útiles cuando se implementan `self-retrievers`, los cuales serán explorados adetalle más adelante.

### Cargar tus documentos

Si prefieres utilizar tus propios documentos, puedes cargarlos a través de la interfaz gráfica de [Docugami](https://www.docugami.com/). Una vez cargados, necesitarás asignar cada uno a un `docset`. Un `docset` es un conjunto de documentos que presentan una estructura análoga. Por ejemplo, todos los contratos de arrendamiento comercial por lo general poseen estructuras similares, por lo que pueden ser agrupados en un único `docset`.

Después de crear tu `docset`, los documentos cargados serán procesados y estarán disponibles para su acceso mediante la API de `Docugami`.

Para recuperar los `ids` de tus documentos y de sus correspondientes `docsets`, puedes ejecutar el siguiente comando:

```bash
curl --header "Authorization: Bearer {YOUR_DOCUGAMI_TOKEN}" \
  https://api.docugami.com/v1preview1/documents
```

Este comando te facilitará el acceso a la información relevante, optimizando así la administración y organización de tus documentos dentro de `Docugami`.

Una vez hayas extraído los `ids` de tus documentos o de los `docsets`, podrás emplearlos para acceder a la información de tus documentos utilizando el `DocugamiLoader` de `Langchain`. Esto te permitirá manipular y gestionar tus documentos dentro de tu aplicación.

In [ ]:
loader = DocugamiLoader(
    docset_id="",
    document_ids=None,
    file_paths=None,
)

papers_docs = loader.load()

In [ ]:
lost_in_the_middle_paper_docs = [
    doc for doc in papers_docs if doc.metadata["source"] == ""
]
for doc in lost_in_the_middle_paper_docs:
    print(doc.metadata["tag"])